In [ ]:
# Set random seed first
import tensorflow as tf
tf.random.set_seed(0)

# Standard libraries
import os
import csv
import pickle
import time
from datetime import datetime
from decimal import Decimal, getcontext

# Scientific computing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    recall_score, precision_score, confusion_matrix, classification_report
)
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KernelDensity

# SciPy and statistics
import scipy.stats as stats
from scipy.stats import (
    normaltest, skew, kurtosis, wasserstein_distance, 
    gaussian_kde, anderson
)
from scipy.optimize import minimize
from statsmodels.stats.diagnostic import lilliefors

# TensorFlow/Keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, MaxPooling3D, AveragePooling2D, 
    Dense, Flatten, BatchNormalization, Activation, 
    Input, Add, ZeroPadding2D, GlobalAveragePooling2D, 
    Dropout, LSTM, ConvLSTM2D, Reshape, TimeDistributed
)
from tensorflow.keras import models, layers, regularizers, optimizers
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, LearningRateScheduler
)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical, plot_model
from scipy.stats import median_abs_deviation, lognorm

sns.set_style("whitegrid", {"grid.color": ".5", "grid.linestyle": ":"})
pd.set_option('display.width', 5000)
pd.set_option('display.max_columns', None)
plt.rcParams['axes.unicode_minus'] = False

In [ ]:

dropout_rates = [0.8,0.6, 0.5, 0.4, 0.3, 0.2, 0.1] #0.5, 0.3, 0.2, 0.1
new_ori_values = []
    
all_trends_dataframes = []

for dropout_rate in dropout_rates:
    print(f"\n{'='*80}")
    print(f"TRAINING WITH DROPOUT RATE: {dropout_rate}")
    
    print(f"{'='*80}\n")
    early_stopping_path = f'logs/early_stopping_point_dropout_{dropout_rate}.csv'
    
    data_metrics = pd.read_csv(f'logs/CIFAR100_ResNet50_dropout_{dropout_rate}.csv')
    print(data_metrics.shape)
    epochs=data_metrics.shape[0]
    print('total epochs:',epochs)
    loss_train=data_metrics['loss']  #training loss
    loss_val=data_metrics['val_loss']  #validating loss
    acc_train=data_metrics['accuracy'] #training accuracy
    acc_val=data_metrics['val_accuracy'] #validating accuracy
    
    recall_train=data_metrics['recall']  #training recall
    recall_val=data_metrics['val_recall']  #validating recall
    precision_train=data_metrics['precision'] #training precision
    precision_val=data_metrics['val_precision'] #validating precision
    
    AUC_train=data_metrics['AUC'] #training AUC
    AUC_val=data_metrics['val_AUC'] #validating AUC
    
    gradients_training_loss1 = np.gradient(loss_train)  # 1st-order gradient of the training loss curve
    gradients_validating_loss1 = np.gradient(loss_val)  # 1st-order gradient of the validating loss curve
    
    gradients_training_loss2 = np.gradient(gradients_training_loss1)   # 2nd-order gradient of the training loss curve
    gradients_validating_loss2 = np.gradient(gradients_validating_loss1)  # 2nd-order gradient of the validating loss curve
    
    
    loss_train=data_metrics['loss']  #training loss
    loss_val=data_metrics['val_loss']  #validating loss
    acc_train=data_metrics['accuracy'] #training accuracy
    acc_val=data_metrics['val_accuracy'] #validating accuracy
    
    recall_train=data_metrics['recall']  #training recall
    recall_val=data_metrics['val_recall']  #validating recall
    precision_train=data_metrics['precision'] #training precision
    precision_val=data_metrics['val_precision'] #validating precision
    
    AUC_train=data_metrics['AUC'] #training AUC
    AUC_val=data_metrics['val_AUC'] #validating AUC
    
    gradients_training_loss1 = np.gradient(loss_train)  # 1st-order gradient of the training loss curve
    gradients_validating_loss1 = np.gradient(loss_val)  # 1st-order gradient of the validating loss curve
    
    gradients_training_loss2 = np.gradient(gradients_training_loss1)   # 2nd-order gradient of the training loss curve
    gradients_validating_loss2 = np.gradient(gradients_validating_loss1)  # 2nd-order gradient of the validating loss curve
    
    loss_training=loss_train.to_numpy()
    loss_validating=loss_val.to_numpy()
    acc_training=acc_train.to_numpy()
    acc_validating=acc_val.to_numpy()
    recall_training=recall_train.to_numpy()
    recall_validating=recall_val.to_numpy()
    precision_training=precision_train.to_numpy()
    precision_validating=precision_val.to_numpy()
    AUC_training=AUC_train.to_numpy()
    AUC_validating=AUC_val.to_numpy()
    

    def wasserstein_flow_lognormal(actual_loss): 
        
        log_loss=np.log(actual_loss + 1e-10)
        
        mu = np.median(log_loss)
        sigma = median_abs_deviation(log_loss, scale='normal')
                
        n = len(actual_loss)
        
        sorted_actual = sorted(actual_loss)
        q_ = np.linspace(0.5/n, 1-0.5/n, n)
               
        def objective_(params):
            mu_opt, sigma_opt = params
            ideal =  stats.lognorm.ppf(q_, s=sigma_opt, scale=np.exp(mu_opt))
            return  np.sum((sorted_actual - ideal)**2) 
        
        result_ = minimize(objective_, [mu, sigma], method='Nelder-Mead')
        mu_opt_, sigma_opt_ = result_.x
        print('best mu and sigma:',mu_opt_, sigma_opt_)
        
        return  stats.lognorm.ppf(q_, s=sigma_opt_, scale=np.exp(mu_opt_))
        
    fit_loss_training=wasserstein_flow_lognormal(loss_training) 

    mean_training = np.mean(loss_training)
    mean_validation = np.mean(loss_validating)
    median_training = np.median(loss_training)
    median_validation = np.median(loss_validating)
    
    print(f"Mean Training: {mean_training}, Mean Validation: {mean_validation}")
    print(f"Median Training: {median_training}, Median Validation: {median_validation}")

    print('difference between validation and training:',mean_validation-mean_training,median_validation-median_training)
   
    fig = plt.figure(figsize=(40,30)) 
    grid = gridspec.GridSpec(9,9)  
    
    with open(early_stopping_path, mode='r') as file:  
            reader = csv.reader(file)  
            stop = int(next(reader)[0])  
    intersection=0
    start=1
    end=len(loss_training)
    epochs1=np.arange(start,len(loss_training)+start)
    
    ax1 = fig.add_subplot(grid[0,0])  
    ax1.plot(loss_training,label='Training',linewidth=1,color='red')
    ax1.plot(loss_validating,label='Validation',linewidth=1,color='green') #,linestyle='dashed'
    ax1.plot(sorted(fit_loss_training,reverse=True),label='Ideal Training',linewidth=1,color='blue',linestyle='dashed') #,linestyle='dashed'
    ax1.set_title('Loss Plot')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Value')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.legend(loc='best',fontsize=10) 
    
    print('***********************************************')
    print('***********************************************')
    d_it= wasserstein_l2(loss_training,fit_loss_training)
    d_iv= wasserstein_l2(loss_validating,fit_loss_training)
    d_vt= wasserstein_l2(loss_training,loss_validating)
    
    print('\n\033[1;31;45m Wasserstein distances:\033[0m\n ')
    print('between training loss and validating loss:')
    print("\033[1;31;45m Wasserstein Distance :(new range) \033[0m", d_vt) 
    #wasserstein_dist(loss_training,loss_validating)
    #wasserstein_dist_point(loss_training,loss_validating)
    print('between training loss and ideal log normal')
    print("\033[1;31;40m Wasserstein Distance :(new range) \033[0m",d_it )
    #wasserstein_dist(loss_training,fit_loss_training)
    #wasserstein_dist_point(loss_training,fit_loss_training)
    print('between ideal log normal and validating loss')
    print("\033[1;31;47m Wasserstein Distance :(new range) \033[0m", d_iv)
    #wasserstein_dist(loss_validating,fit_loss_training)
    #wasserstein_dist_point(loss_validating,fit_loss_training)
    
    
    rank, assessment, explanation = evaluate_overfitting(d_it, d_iv, d_vt)
    if mean_validation-mean_training<0 and median_validation-median_training<0:
        new_ORI=0
    else:
        new_ORI=max(0,1-d_it/d_iv)
    new_ori_values.append(new_ORI)
    
    print(f"Rank Pattern: {rank}")
    print(f"Overfitting Level: {assessment}")
    print(f"Explanation: {explanation}")
    
    print('\033[1;35;44m Wasserstein distance trend: train~validating\033[0m',calculate_wasserstein_and_ori_trend(loss_training,loss_validating,fit_loss_training,start,end)[0])  #train~val
    print('\033[1;35;45m Wasserstein distance trend:ideal train~training\033[0m',calculate_wasserstein_and_ori_trend(loss_training,loss_validating,fit_loss_training,start,end)[1])  # ideal train~val
    print('\033[1;35;46m Wasserstein distance trend: Ideal traing~validatning\033[0m',calculate_wasserstein_and_ori_trend(loss_training,loss_validating,fit_loss_training,start,end)[2]) # train~ideal train
    print('\033[1;35;47m NEW ORI INDEX\033[0m',new_ORI) # train~ideal train
    print(new_ori_values)
   
    w1, w2, w3, ori_trend = calculate_wasserstein_and_ori_trend(loss_training,loss_validating,fit_loss_training,start,end)#Wasserstein_trend1(start, end)
    
    new_ORI = ori_trend[-1]
    print(f"Calculated ORI: {new_ORI:.4f}")

    trends_df = pd.DataFrame({
            'epoch': range(1, len(loss_training) + 1),
            'dropout_rate': dropout_rate,
            'ori_trend': ori_trend
        })
    all_trends_dataframes.append(trends_df)
    fig, axs = plt.subplots(1, 4, figsize=(20, 4.5))
    
    ax = axs[0]
    ax.plot(loss_training, label='Training', linewidth=1.5, color='red')
    ax.plot(loss_validating, label='Validation', linewidth=1.5, color='green')
    ax.plot(sorted(fit_loss_training,reverse=True), label='Ideal Training', linewidth=1.5, color='blue', linestyle='--')
    ax.set_title('Loss Curves')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss Value')
    ax.axvline(stop, linestyle='--', color='skyblue', label=f'Early Stop (Epoch {stop})')
    ax.annotate(' ES: epoch '+str(stop), xy=(stop-8, max(max(loss_training),max(loss_validating))*0.7),xytext=(stop-8, max(max(loss_training),max(loss_validating))*0.7),fontsize=10 )   #set text annotation for intersection line 
    ax.legend(loc='best')
    ax.grid(True, which='both', linestyle=':', linewidth=0.5)

    ax = axs[1]
    sns.kdeplot(loss_training, color='red', label='Training', ax=ax)
    sns.kdeplot(loss_validating, color='green', label='Validation', ax=ax)
    sns.kdeplot(fit_loss_training, color='blue', label='Ideal Training', linestyle='--', ax=ax)
    ax.set_title('Density of Loss Values')
    ax.set_xlabel('Loss Value')
    ax.set_ylabel('Density')
    ax.legend(loc='best')
    ax.grid(True, which='both', linestyle=':', linewidth=0.5)
   
    ax = axs[2]
    ax.plot(w1, color='red', linewidth=2.0, linestyle='-', label='WD(Train, Valid)')
    ax.plot(w2, color='black', linewidth=2.0, linestyle='--', label='WD(Train, Ideal)')
    ax.plot(w3, color='green', linewidth=2.0, linestyle='-', label='WD(Valid, Ideal)')
    ax.set_title('Wasserstein Distance Trends')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Distance')
    #ax.axvline(stop, linestyle='--', color='#40BEE6')
    ax.axvline(stop, linestyle='--', color='skyblue', label=f'Early Stop (Epoch {stop})')
    ax.annotate(' ES: epoch '+str(stop), xy=(stop-8, max(max(w2),max(w3))*0.7),xytext=(stop-8,max(max(w2),max(w3))*0.7),fontsize=10 )   #set text annotation f
    ax.legend(loc='best')
    ax.grid(True, which='both', linestyle=':', linewidth=0.5)
    

    ax = axs[3]
    ax.plot(ori_trend,color='#732BF5',linewidth =2.0,linestyle='solid',label='ORI Trend')
    ax .set_title('ORI Trend')
    ax .set_xlabel('Epoch')
    ax.set_ylabel('Values')
    #ax9 .set_xlim(150,200)
    ax.set_ylim(-0.01,1)
    ax.vlines(stop, 0,1,linestyles='dashdot', colors='skyblue')  #plot earlystopping line
    
    ax.annotate(' ES: epoch '+str(stop), xy=(stop-8, max(ori_trend)*0.5),xytext=(stop-8,  max(ori_trend)*0.5),fontsize=10 )   #set text annotation for intersection line 
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend(loc='best') 

    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f'logs/CIFAR100_ResNet50_dropout_{dropout_rate}.png', dpi=300)
    plt.show()
if all_trends_dataframes:
    final_trends_df = pd.concat(all_trends_dataframes, ignore_index=True)
    output_csv_path = 'results/all_per_epoch_ori_trends.csv'
    final_trends_df.to_csv(output_csv_path, index=False)
    print(f"\nSaved all per-epoch ori_trends to '{output_csv_path}'")
    plot_all_ori_trends(csv_path=output_csv_path)
else:
    print("\nNo ori_trend data was collected. Skipping CSV saving and plotting.")
if len(new_ori_values) == len(dropout_rates):
    ori_df = pd.DataFrame({
        'dropout_rate': dropout_rates,
        'new_ori': new_ori_values
    })
    
    # Define the output path (e.g., in 'results' directory to match other outputs)
    ori_output_csv_path = 'results/new_ori_values_by_dropout.csv'
    
    # Save to CSV
    ori_df.to_csv(ori_output_csv_path, index=False)
    print(f"\nSaved new_ori_values for all dropout rates to '{ori_output_csv_path}'")
else:
    print("\nWarning: Mismatch between dropout_rates and new_ori_values lengths. Skipping CSV save.")

